# Data Uploader

This notebook demonstrates the process of uploading EDF files data to Delta Lake and OpenStack Swift for long-term storage. 

It also includes the setup and execution of the data upload process, as well as querying the uploaded data for analysis.

## Starting the servers:
To launch the server, open the Docker Desktop app and open a terminal in the root of the [DiveDB repo](https://github.com/ecophysviz-lab/DiveDB). Then run the following command:
```bash
$ make up
```
This command will launch the Django server, Postgres database, and Jupyter server using the environment variables defined in the `.env` file accross all containers. 

**Ensure you have the same env variables in Pylogger**

#### Understanding expected file paths:
DiveDB expects the following paths to be set in the `.env` file:
- `CONTAINER_DATA_PATH`
- `LOCAL_DATA_PATH`
- `HOST_DELTA_LAKE_PATH`
- `CONTAINER_DELTA_LAKE_PATH`
- `HOST_FILE_STORAGE_PATH`
- `CONTAINER_FILE_STORAGE_PATH`

These paths are used to mount the Delta Lake and file storage to the containers. The "LOCAL_" and "HOST_" paths can be wherever makes sense for your local machine. The "CONTAINER_" paths are the paths that the containers expect. We recommend you keep the "CONTAINER_" paths as they are in the `.env.example` file.

#### When is the server ready?
There are 3 processes that need to be running for the server to be ready:
1. The Django server (`web`)
2. The Postgres database (`metadata_database`)
3. The Jupyter server (`jupyter`)

Jupyter is almost always the last to start up. You'll know it's ready when you see the following logs in the terminal:
```bash
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] Serving notebooks from local directory: /app
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] Jupyter Server 2.14.2 is running at:
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] http://e29d05e13fd0:8888/jupyter/tree
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp]     http://127.0.0.1:8888/jupyter/tree
```

## Preparing to upload data:
There are two aspects to any data upload:
1. A table containing measurements and time
    - This can be an EDF, CSV, or a data
2. The metadata for the measurements
    - This describes the context of the measurements using the following fields:
        - animal
        - deployment
        - logger
        - recording

There are several ways to define your metadata. 

#### Option 1: Supplied Metadata CSV File
If you have a metadata file, you can use the `metadata_map` dictionary to map the columns in the CSV file to the corresponding mode. You can then pass the metadata file to the `upload_edf` function which will interactively extract the metadata for your measurements using the data in the metadata file and in the Metadata Database.

#### Option 2: Interactive Selection (in the works)
If you provide no metadata, you can use the `upload_edf` function to interactively select the metadata for your measurements using the data in the Metadata Database.

#### Option 3: Supplied Metadata Dictionary
If you know the metadata for your measurements, you can pass a dictionary to the `upload_edf` function. The dictionary should represent metadata existing in the Metadata database and contain the following fields:
- animal: The animal ID
- deployment: The deployment name
- logger: The logger ID
- recording: The recording name


### Uploading EDFs
The `edf_file_paths` list contains the paths to the EDF files that we want to upload. It can point to files on your local machine or on a remote server.
In this example, the files are located in the ../data/files/ directory and are named test12_Wednesday_05_DAY1_PROCESSED.edf and test12_Wednesday_05_DAY2_PROCESSED.edf.

The `metadata_file_path` variable holds the path to the CSV file containing metadata for the EDF files (named Sleep Study Metadata.csv). 

The `metadata_map` dictionary is used to map the columns in the CSV metadata file to the corresponding mode. The keys in the dictionary represent the fields in the database, and the values represent the column names in the CSV file. For example:
- "animal" maps to the "Nickname" column in the CSV file.
- "deployment" maps to the "Deployment" column in the CSV file.
- "logger" maps to the "Logger Used" column in the CSV file.
- "recording" maps to the "Recording ID" column in the CSV file.

The upload_edf function will perform the following: 
- use the metadata map to extract the metadata for your measurements using the data in the metadata file and in the Metadata Database
- upload copies of the EDF files to OpenStack Swift
- upload the measurements to Delta Lake (by default, 10M measurements per batch)

The process takes between 5-10 minutes to complete per gigabyte. (*note: we can speed this up by parellizing the upload process*)


In [6]:
# Option 1: Supplied Metadata CSV File

import os

# Allow Django to run with async unsafe to run outside of Django server
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from DiveDB.services.data_uploader import DataUploader


data_uploader = DataUploader()


edf_file_paths = [
    "../data/files/test12_Wednesday_05_DAY1_PROCESSED.edf",
    "../data/files/test12_Wednesday_05_DAY2_PROCESSED.edf"
]

metadata_file_path = "../data/files/Sleep Study Metadata.csv"

metadata_map = {
    "animal": "Nickname",
    "deployment": "Deployment",
    "logger": "Logger Used",
    "recording": "Recording ID"
}

data_uploader.upload_edf(edf_file_paths, metadata_file_path, metadata_map)

RuntimeError: populate() isn't reentrant

In [2]:
# Option 3: Supplied Metadata

import os

# Allow Django to run with async unsafe to run outside of Django server
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from DiveDB.services.data_uploader import DataUploader


data_uploader = DataUploader()


edf_file_paths = [
    "../data/files/test12_Wednesday_05_DAY1_PROCESSED.edf",
    "../data/files/test12_Wednesday_05_DAY2_PROCESSED.edf"
]
metadata = {
    "animal": "test12_Wednesday",
    "deployment": "2024-06-06_boat-001a",
    "logger": "NL-02",
    "recording": "2019-10-25_mian-001a_NL-02_001"
}

data_uploader.upload_edf(edf_file_paths, metadata=metadata)

RuntimeError: populate() isn't reentrant

### Uploading NetCDF files
The `netcdf_file_path` list contains the paths to the NetCDF files that we want to upload. It can point to files on your local machine or on a remote server.
In this example, the file is located in the ../data/files/ directory and is named deployment_data.nc.

The upload_netcdf function will perform the following: 
- use the provided metadata dictionary to extract the metadata for your measurements
- upload copies of the NetCDF files to OpenStack Swift
- upload the measurements to Delta Lake

The process takes between 30 secs to 1 min to complete per gigabyte — about 2/3rds of the time is used to upload the files to OpenStack Swift. (*note: we can speed this up by parellizing the upload process*)

In [3]:
import os

# Allow Django to run with async unsafe to run outside of Django server
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from DiveDB.services.data_uploader import DataUploader

data_uploader = DataUploader()

metadata = {
    "animal": "mian-001",
    "deployment": "2024-06-18_oror-001a",
    "logger": "NL-01",
    "recording": "2024-06-06_boat-001a_WC75_002"
}

data_uploader.upload_netcdf("../data/2024-01-16_oror-002a/outputs/deployment_data.nc", metadata)

Creating file record for deployment_data.nc and uploading to OpenStack...
Processing 9 variables in the netCDF file.


Processing variables: 100%|██████████| 9/9 [00:21<00:00,  2.42s/it]

Upload complete.
